In [2]:
# # pytorch 
# ! pip3 install torch torchvision torchaudio

# # segmentation dependencies 
# !pip install segmentation-models-pytorch
# !pip install pytorch-lightning
# !pip install pandas 
# !pip install rasterio

In [1]:

import torch
import numpy as np
import PIL
from tqdm import tqdm
from PIL import Image
import pytorch_lightning as pl
import segmentation_models_pytorch as smp
from matplotlib import pyplot as plt
import sys  
import os
from pathlib import Path
from matplotlib.pyplot import imshow
from matplotlib import pyplot as plt
import numpy as np

# Set Root Directory
root = Path('/mnt/c/Users/Samarth Negi/Desktop/soccerEye')
sys.path.insert(0, str(root))


from IPython.display import Video
import cv2
import methods 
import utils


In [2]:
# Set Dataset Path
dataset = Path('../dataset/df-bl/')
dataset = dataset.resolve()
print(f'Dataset Location : {dataset}')
dataset_files = utils.util_searching_all_files(dataset)
print(f'Number of files in dataset : {len(dataset_files)}')

Dataset Location : /mnt/c/Users/Samarth Negi/Desktop/soccerEye/dataset/df-bl
Number of files in dataset : 12


In [3]:
videoPick = np.random.randint(len(dataset_files))
print(f'Video Picked (index) : {videoPick}')
cap = cv2.VideoCapture(str(dataset_files[videoPick]))

Video Picked (index) : 3


In [4]:
total_frames = cap.get(7)
print(total_frames)

87732.0


In [77]:
skip = 1000

In [78]:
FRAMELIST = []
for framePick in tqdm(range(0,int(total_frames),skip)):
    cap.set(1, framePick)
    ret, frame = cap.read()
    
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    im_pil = Image.fromarray(img)

    
    FRAMELIST.append(im_pil)

100%|███████████████████████████████████████████████████████████████████████████████████| 88/88 [00:34<00:00,  2.54it/s]


In [79]:
OUTFRAMES = methods.P_IF_4_FPN(FRAMELIST,
                      '../weights/FPN_FieldMask_1/checkpoints/epoch=49-step=600.ckpt',
                      inputType='list' )

In [80]:
FINAL = []

for ind,FRAME in enumerate(FRAMELIST):
    
    frame_np = np.asarray(FRAME)    
    frame_np_r = cv2.resize(frame_np, (256,256), interpolation = cv2.INTER_AREA)
    
    outframe_normal = cv2.normalize(OUTFRAMES[ind], None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
    outframe_normal_rgb = cv2.cvtColor(outframe_normal, cv2.COLOR_GRAY2BGR)
    
    final_output = np.concatenate([frame_np_r,outframe_normal_rgb],axis=1)
    
    FINAL.append(final_output)

In [81]:
w = FINAL[0].shape[1]
h = FINAL[0].shape[0]

In [82]:
out = cv2.VideoWriter('output.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 10, (w,h))

In [83]:
out = cv2.VideoWriter('output.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (w,h))

In [84]:
for framePick in FINAL:
    out.write(framePick)

In [85]:
out.release()